An advanced Multi-layer perceptron approach to the classification of skin cancer images is shown here. Lets load the requisite libraries.

In [ ]:
import numpy as np
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from keras.utils import np_utils
%matplotlib inline

Lets load the dataset.

In [ ]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
dataframe = read_csv("Book1.csv", header=None)
dataset = dataframe.values
X = dataset[:,0:15].astype(float)
Y = dataset[:,15]

Lets perform One hot encoding or create dummy variables from a categorical variable (class) and encode class values as integers. 

In [ ]:
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
dummy_y = np_utils.to_categorical(encoded_Y)

Lets define a baseline model: A simple fully connected network with one hidden layer that contains 15 neurons, the same number of inputs. The hidden layer uses a rectifier activation function. Because we used a one-hot encoding for our dataset, the output layer must create 3 output values, one for each class. The output value with the largest value will be taken as the class predicted by the model. The network topology of this simple one-layer neural network can be summarized as:

15 inputs -> [15 hidden nodes] -> 3 outputs

We use a sigmoid activation function in the output layer to ensure the output values are in the range of 0 and 1 and may be used as predicted probabilities. Finally, the network uses ADAM gradient descent optimization algorithm with a logarithmic loss function, which is called categorical crossentropy. Initially we can define a model and test its performance with k-fold cross-validation (k=10) and make changes to the model accordingly to get better accuracy. Later we can fine-tune the performance with GridSearchCV to find the optimal values for the hyper-parameters.

In [ ]:
def baseline_model():
    # create model
    model = Sequential()
    model.add(Dense(50, input_dim=15, kernel_initializer='normal', activation='relu'))
    model.add(Dense(3, kernel_initializer='normal', activation='sigmoid'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

Improve Performance With Data Preparation:
It is a good practice to prepare your data before modeling. Neural network models are especially suitable to having consistent input values, both in scale and distribution. An effective data preparation scheme for tabular data when building neural network models is Min_Max Scaling and standardization. This is where the data is rescaled such all values of the attributes lies between 0 and 1 and that the mean value for each attribute is 0 and the standard deviation is 1. This preserves Gaussian and Gaussian-like distributions whilst normalizing the central tendencies for each attribute. We can use scikit-learn to perform the Min-Max scaling and standardization of our dataset using the Min_Max Scaler and StandardScaler class. 

Rather than performing the standardization on the entire  dataset, it is good practice to train the standardization procedure on the training data within the pass of a cross-validation run and to use the trained standardization instance to prepare the unseen test fold. This makes standardization a step in model preparation in the cross-validation process and it prevents the algorithm having knowledge of unseen data during evaluation, knowledge that might be passed from the data preparation scheme like a crisper distribution. We can achieve this in scikit-learn using a Pipeline class. The pipeline is a wrapper that executes one or more models within a pass of the cross-validation procedure. Here, we can define a pipeline with the StandardScaler followed by our neural network model. 

We can now create our Classifier for use in scikit-learn and pass arguments in the construction of the KerasClassifier class that will be passed on to the fit() function internally used to train the neural network. Here, we pass the number of epochs epochs as 200 and batch size as 5 to use when training the model. Debugging is also turned on when training by setting verbose to 1.

In [ ]:
estimators = []
estimators.append(('MinMaxScale', MinMaxScaler()))
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=baseline_model, epochs=100,
batch_size=5, verbose=1)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=2, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

The function that we specify to the build_fn argument when creating the KerasClassifier wrapper can also take arguments. We can use these arguments to further customize the construction of the model. We use a grid search to evaluate different configurations for our neural network model and report on the combination that provides the best estimated performance. The baseline_model() function is defined to take two arguments optimizer and init, both of which must have default values. This will allow us to evaluate the effect of using different optimization algorithms and weight initialization schemes for our network. After creating our model, we define arrays of values for the parameter we wish to search, specifically:

1. Optimizers for searching different weight values.
2. Initializers for preparing the network weights using different schemes.
3. Number of epochs for training the model for different number of exposures to the training dataset.
4. Batches for varying the number of samples before weight updates.

The options are specified into a dictionary and passed to the configuration of the GridSearchCV scikit-learn class. This class will evaluate a version of our neural network model for each combination of parameters (2 x 3 x 3 x 3) for the combinations of optimizers, initializations,epochs and batches). Each combination is then evaluated using the default of 3-fold stratified
cross-validation.That is a lot of models and a lot of computation. This is not a scheme that you want to use lightly because of the time it will take to compute. It may be useful for you to design small experiments with a smaller subset of your data that will complete in a reasonable time. This experiment is reasonable in this case because of the small network and the small dataset (only 1500 instances and 15 attributes). Finally, the performance and combination of configurations for the best model are displayed, followed by the performance of all combinations of parameters.

In [ ]:
# Function to create model, required for KerasClassifier
def create_model(optimizer='rmsprop', init='glorot_uniform'):
    # create model
    model = Sequential()
    model.add(Dense(50, input_dim=15, kernel_initializer=init, activation='relu'))
    model.add(Dense(3, kernel_initializer=init, activation='sigmoid'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

# load dataset
dataframe = read_csv("Book1.csv", header=None)
dataset = dataframe.values
X = dataset[:,0:15].astype(float)
Y = dataset[:,15]

# create model
model = KerasClassifier(build_fn=create_model, verbose=1)

# grid search epochs, batch size and optimizer
optimizers = ['rmsprop', 'adam']
inits = ['glorot_uniform', 'normal', 'uniform']
epochs = [50, 100, 150, 200]
batches = [5, 10, 15, 20, 25, 30]
param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches, init=inits)
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(X, dummy_y)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

JSON is a simple file format for describing data hierarchically. Keras provides the ability to describe any model using JSON format with a to_json() function. This can be saved to file and later loaded via the model_from_json() function that will create a new model from the JSON specication. The weights are saved directly from the model using the save_weights() function and later loaded using the symmetrical load_weights() function. Lets train and evaluate our model on the dataset. The model structure is then converted to JSON format and written to model.json in the local directory. The network weights are written to model.h5 in the local directory. The model and weight data is loaded from the saved files and a new model is created. It is important to compile the loaded model before it is used. This is so that predictions made using the model can use the appropriate efficient computation from the Keras backend. The model is evaluated in the same way printing the same evaluation score. 

In [ ]:
# fix random seed for reproducibility
numpy.random.seed(7)

# load data
dataframe = read_csv("Book1.csv", header=None)
dataset = dataframe.values
X = dataset[:,0:15].astype(float)
Y = dataset[:,15]

#standardize data with minmaxscaling and standard scaling
scaler = MinMaxScaler().fit(X)
rescaledX = scaler.transform(X)
scaler = StandardScaler().fit(X)
rescaledX = scaler.transform(X)

# create model
model = Sequential()
model.add(Dense(50, input_dim=15, kernel_initializer='normal', activation='relu'))
model.add(Dense(3, kernel_initializer='normal', activation='sigmoid'))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
model.fit(rescaledX, dummy_y, epochs=150, batch_size=10, verbose=1)

# evaluate the model
scores = model.evaluate(rescaledX, dummy_y, verbose=1)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
    
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

In [ ]:
# later...
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate(rescaledX, dummy_y, verbose=1)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

You can also save Your Neural Network Model to YAML: This is much the same as the above JSON, except the YAML format is used for the model specication. The model is described using YAML, saved to file model.yaml and later loaded into a new model via the model from yaml() function. Weights are handled in the same way as above in HDF5 format as model.h5.

#serialize model to YAML
model_yaml = model.to_yaml()
with open("model.yaml", "w") as yaml_file:
    yaml_file.write(model_yaml)

#serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

#later...
#load YAML and create model
yaml_file = open('model.yaml', 'r')
loaded_model_yaml = yaml_file.read()
yaml_file.close()
loaded_model = model_from_yaml(loaded_model_yaml)

#load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

#evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate(X, Y, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))'''

Checkpoint Neural Network Model Improvements: 
A good use of checkpointing is to output the model weights when an improvement is observed during training. A simpler checkpoint strategy is to save the model weights to the same file, if and only if the validation accuracy improves. The model weights are written to the file weights.best.hdf5 only if the classication accuracy of the model on the validation dataset improves over the best seen so far.